### Import Libraries

In [1]:
import numpy as np 
import pandas as pd 
import nltk
import os
import gc
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
#pd.set_option('display.max_colwidth',100)
pd.set_option('display.max_colwidth', -1)

Using TensorFlow backend.


In [2]:
help(LSTM)

Help on class LSTM in module keras.layers.recurrent:

class LSTM(RNN)
 |  Long Short-Term Memory layer - Hochreiter 1997.
 |  
 |  # Arguments
 |      units: Positive integer, dimensionality of the output space.
 |      activation: Activation function to use
 |          (see [activations](../activations.md)).
 |          Default: hyperbolic tangent (`tanh`).
 |          If you pass `None`, no activation is applied
 |          (ie. "linear" activation: `a(x) = x`).
 |      recurrent_activation: Activation function to use
 |          for the recurrent step
 |          (see [activations](../activations.md)).
 |          Default: hard sigmoid (`hard_sigmoid`).
 |          If you pass `None`, no activation is applied
 |          (ie. "linear" activation: `a(x) = x`).
 |      use_bias: Boolean, whether the layer uses a bias vector.
 |      kernel_initializer: Initializer for the `kernel` weights matrix,
 |          used for the linear transformation of the inputs.
 |          (see [initializ

In [3]:
gc.collect()

7

In [4]:
gc.collect()

0

#### Load the dataset

In [5]:
train = pd.read_csv("input/train.tsv", sep='\t')
print (train.shape)
train.head()

(156060, 4)


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [6]:
test = pd.read_csv("input/test.tsv", sep='\t')
print (test.shape)
test.head()

(66292, 3)


,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine effort .
1,156062,8545,An intermittently pleasing but mostly routine effort
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [7]:
sub = pd.read_csv("input/sampleSubmission.csv")
print (sub.shape)
sub.head()

(66292, 2)


,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


#### Adding sentiment column to test data, join the train and test for preprocessing

In [8]:
test['Sentiment'] = -99
test.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,156061,8545,An intermittently pleasing but mostly routine effort .,-99
1,156062,8545,An intermittently pleasing but mostly routine effort,-99
2,156063,8545,An,-99
3,156064,8545,intermittently pleasing but mostly routine effort,-99
4,156065,8545,intermittently pleasing but mostly routine,-99


In [9]:
df = pd.concat([train, test], ignore_index=True)
print df.shape
df.tail()

(222352, 4)


,PhraseId,SentenceId,Phrase,Sentiment
222347,222348,11855,"A long-winded , predictable scenario .",-99
222348,222349,11855,"A long-winded , predictable scenario",-99
222349,222350,11855,"A long-winded ,",-99
222350,222351,11855,A long-winded,-99
222351,222352,11855,predictable scenario,-99


In [10]:
del train, test
gc.collect()

120

#### Cleaning the reviews

In [11]:
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer

from string import punctuation
import re

nltk.download('wordnet')

stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /home/suraj/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
def clean_review(review_col):
    review_corpus=[]
    for i in range(0,len(review_col)):
        review=str(review_col[i])
        review=re.sub('[^a-zA-Z]',' ',review) # remove all the numerical values
        #review=[stemmer.stem(w) for w in word_tokenize(str(review).lower())]
        review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
        review=' '.join(review)
        review_corpus.append(review)
        if i % 10000 == 0:
            print "-----------------Completed " + str(i) + " phrases"
    return review_corpus

In [13]:
df['clean_review']=clean_review(df.Phrase.values)
df.head()

-----------------Completed 0 phrases
-----------------Completed 10000 phrases
-----------------Completed 20000 phrases
-----------------Completed 30000 phrases
-----------------Completed 40000 phrases
-----------------Completed 50000 phrases
-----------------Completed 60000 phrases
-----------------Completed 70000 phrases
-----------------Completed 80000 phrases
-----------------Completed 90000 phrases
-----------------Completed 100000 phrases
-----------------Completed 110000 phrases
-----------------Completed 120000 phrases
-----------------Completed 130000 phrases
-----------------Completed 140000 phrases
-----------------Completed 150000 phrases
-----------------Completed 160000 phrases
-----------------Completed 170000 phrases
-----------------Completed 180000 phrases
-----------------Completed 190000 phrases
-----------------Completed 200000 phrases
-----------------Completed 210000 phrases
-----------------Completed 220000 phrases


,PhraseId,SentenceId,Phrase,Sentiment,clean_review
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1,a series of escapade demonstrating the adage that what is good for the goose is also good for the gander some of which occasionally amuses but none of which amount to much of a story
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2,a series of escapade demonstrating the adage that what is good for the goose
2,3,1,A series,2,a series
3,4,1,A,2,a
4,5,1,series,2,series


In [14]:
df.head(20)

,PhraseId,SentenceId,Phrase,Sentiment,clean_review
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1,a series of escapade demonstrating the adage that what is good for the goose is also good for the gander some of which occasionally amuses but none of which amount to much of a story
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2,a series of escapade demonstrating the adage that what is good for the goose
2,3,1,A series,2,a series
3,4,1,A,2,a
4,5,1,series,2,series
5,6,1,of escapades demonstrating the adage that what is good for the goose,2,of escapade demonstrating the adage that what is good for the goose
6,7,1,of,2,of
7,8,1,escapades demonstrating the adage that what is good for the goose,2,escapade demonstrating the adage that what is good for the goose
8,9,1,escapades,2,escapade
9,10,1,demonstrating the adage that what is good for the goose,2,demonstrating the adage that what is good for the goose


#### Separating training and testing data

In [15]:
df_train = df[df.Sentiment != -99]
df.shape

(222352, 5)

In [16]:
df_test=df[df.Sentiment==-99]
df_test.drop('Sentiment',axis=1,inplace=True)
print(df_test.shape)
df_test.head()

(66292, 4)


,PhraseId,SentenceId,Phrase,clean_review
156060,156061,8545,An intermittently pleasing but mostly routine effort .,an intermittently pleasing but mostly routine effort
156061,156062,8545,An intermittently pleasing but mostly routine effort,an intermittently pleasing but mostly routine effort
156062,156063,8545,An,an
156063,156064,8545,intermittently pleasing but mostly routine effort,intermittently pleasing but mostly routine effort
156064,156065,8545,intermittently pleasing but mostly routine,intermittently pleasing but mostly routine


In [17]:
del df
gc.collect()

120

In [18]:
train_text=df_train.clean_review.values
test_text=df_test.clean_review.values
target=df_train.Sentiment.values
y=to_categorical(target)
print(train_text.shape,target.shape,y.shape)

((156060,), (156060,), (156060, 5))


In [19]:
for i in train_text[:5]: print i

a series of escapade demonstrating the adage that what is good for the goose is also good for the gander some of which occasionally amuses but none of which amount to much of a story
a series of escapade demonstrating the adage that what is good for the goose
a series
a
series


In [20]:
target[:5]

array([1, 2, 2, 2, 2])

In [21]:
y[:5]

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [22]:
help(train_test_split)

Help on function train_test_split in module sklearn.model_selection._split:

train_test_split(*arrays, **options)
    Split arrays or matrices into random train and test subsets
    
    Quick utility that wraps input validation and
    ``next(ShuffleSplit().split(X, y))`` and application to input data
    into a single call for splitting (and optionally subsampling) data in a
    oneliner.
    
    Read more in the :ref:`User Guide <cross_validation>`.
    
    Parameters
    ----------
    *arrays : sequence of indexables with same length / shape[0]
        Allowed inputs are lists, numpy arrays, scipy-sparse
        matrices or pandas dataframes.
    
    test_size : float, int, None, optional
        If float, should be between 0.0 and 1.0 and represent the proportion
        of the dataset to include in the test split. If int, represents the
        absolute number of test samples. If None, the value is set to the
        complement of the train size. By default, the value is set 

In [23]:
X_train_text,X_val_text,y_train,y_val=train_test_split(train_text,y,test_size=0.2,stratify=y,random_state=123)
print(X_train_text.shape,y_train.shape)
print(X_val_text.shape,y_val.shape)

((124848,), (124848, 5))
((31212,), (31212, 5))


In [24]:
all_words=' '.join(X_train_text)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
num_unique_word

13732

In [25]:
dist

FreqDist({u'the': 41465, u'a': 36291, u'of': 26383, u'and': 25755, u'to': 18324, u'it': 15166, u's': 13981, u'in': 11208, u'is': 10752, u'that': 9835, ...})

#### Finding the max length of review

In [26]:
r_len=[]
for text in X_train_text:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)
    
MAX_REVIEW_LEN=np.max(r_len)
MAX_REVIEW_LEN

48

### Building Keras LSTM Model

In [27]:
max_features = num_unique_word
max_words = MAX_REVIEW_LEN
batch_size = 128
epochs = 3
num_classes=5

#### Tokenize the text

In [28]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train_text))
X_train = tokenizer.texts_to_sequences(X_train_text)
X_val = tokenizer.texts_to_sequences(X_val_text)
X_test = tokenizer.texts_to_sequences(test_text)

In [29]:
X_train[:5]

[[10, 100, 702, 19, 5, 141, 6, 2, 6022, 3, 380],
 [11771],
 [171, 471, 16],
 [618, 669],
 [56, 349]]

#### Sequence Padding

In [30]:
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_val = sequence.pad_sequences(X_val, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
print(X_train.shape,X_val.shape,X_test.shape)

((124848, 48), (31212, 48), (66292, 48))


### 1. LSTM Model

In [31]:
model1=Sequential()
model1.add(Embedding(max_features,100,mask_zero=True))
model1.add(LSTM(64,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model1.add(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model1.add(Dense(num_classes,activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         1373200   
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 64)          42240     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 165       
Total params: 1,428,021
Trainable params: 1,428,021
Non-trainable params: 0
_________________________________________________________________


In [32]:
# %%time
# history1=model1.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

In [33]:
# y_pred1=model1.predict_classes(X_test,verbose=1)

### 2. CNN

In [34]:
model2= Sequential()
model2.add(Embedding(max_features,100,input_length=max_words))
model2.add(Dropout(0.2))

model2.add(Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model2.add(GlobalMaxPooling1D())

model2.add(Dense(128,activation='relu'))
model2.add(Dropout(0.2))

model2.add(Dense(num_classes,activation='softmax'))


model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 48, 100)           1373200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 100)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 48, 64)            19264     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 645       
Total para

In [35]:
help(GlobalMaxPooling1D)

Help on class GlobalMaxPooling1D in module keras.layers.pooling:

class GlobalMaxPooling1D(_GlobalPooling1D)
 |  Global max pooling operation for temporal data.
 |  
 |  # Input shape
 |      3D tensor with shape: `(batch_size, steps, features)`.
 |  
 |  # Output shape
 |      2D tensor with shape:
 |      `(batch_size, features)`
 |  
 |  Method resolution order:
 |      GlobalMaxPooling1D
 |      _GlobalPooling1D
 |      keras.engine.base_layer.Layer
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  call(self, inputs)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from _GlobalPooling1D:
 |  
 |  __init__(self, **kwargs)
 |  
 |  compute_output_shape(self, input_shape)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from keras.engine.base_layer.Layer:
 |  
 |  __call__(self, inputs, **kwargs)
 |      Wrapper around self.call(), for handling internal references

In [36]:
%%time
history2=model2.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Train on 124848 samples, validate on 31212 samples
Epoch 1/3
124848/124848 [==============================] - 67s 540us/step - loss: 1.0014 - acc: 0.5994 - val_loss: 0.8502 - val_acc: 0.6523
Epoch 2/3
124848/124848 [==============================] - 67s 533us/step - loss: 0.7842 - acc: 0.6779 - val_loss: 0.8244 - val_acc: 0.6593
Epoch 3/3
124848/124848 [==============================] - 67s 535us/step - loss: 0.7052 - acc: 0.7070 - val_loss: 0.8079 - val_acc: 0.6624
CPU times: user 7min 30s, sys: 48.4 s, total: 8min 18s
Wall time: 3min 21s


In [37]:
y_pred2=model2.predict_classes(X_test, verbose=1)

66292/66292 [==============================] - 8s 118us/step


### 3. CNN + GRU

In [38]:
model3= Sequential()
model3.add(Embedding(max_features,100,input_length=max_words))
model3.add(Conv1D(32,kernel_size=3,padding='same',activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Dropout(0.25))
model3.add(GRU(64,return_sequences=True))
model3.add(Dropout(0.3))
model3.add(Flatten())
model3.add(Dense(64,activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(num_classes,activation='softmax'))
model3.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 48, 100)           1373200   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 48, 32)            9632      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 24, 32)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 24, 32)            0         
_________________________________________________________________
gru_1 (GRU)                  (None, 24, 64)            18624     
_________________________________________________________________
dropout_4 (Dropout)          (None, 24, 64)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1536)              0         
__________

In [39]:
gc.collect()

296

In [40]:
# %%time
# history3=model3.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

In [41]:
# y_pred3=model3.predict_classes(X_test, verbose=1)

### 4. Bidirectional GRU

In [42]:
model4 = Sequential()

model4.add(Embedding(max_features, 100, input_length=max_words))
model4.add(SpatialDropout1D(0.25))
model4.add(Bidirectional(GRU(128)))
model4.add(Dropout(0.5))

model4.add(Dense(5, activation='softmax'))
model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 48, 100)           1373200   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 48, 100)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               175872    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 1285      
Total params: 1,550,357
Trainable params: 1,550,357
Non-trainable params: 0
_________________________________________________________________


In [43]:
%%time
history4=model4.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Train on 124848 samples, validate on 31212 samples
Epoch 1/3
110080/124848 [=========================>....] - ETA: 36s - loss: 1.0243 - acc: 0.5875

KeyboardInterrupt: 

In [ ]:
y_pred4=model4.predict_classes(X_test, verbose=1)

###   5. Glove Word Embedding

In [46]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')
    
def get_embed_mat(EMBEDDING_FILE, max_features, embed_dim):
    # word vectors
    embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))
    print('Found %s word vectors.' % len(embeddings_index))

    # embedding matrix
    word_index = tokenizer.word_index
    num_words = min(max_features, len(word_index) + 1)
    all_embs = np.stack(embeddings_index.values()) #for random init
    embedding_matrix = np.random.normal(all_embs.mean(), all_embs.std(), 
                                        (num_words, embed_dim))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    max_features = embedding_matrix.shape[0]
    
    return embedding_matrix


In [47]:
# embedding matrix
EMBEDDING_FILE = 'input/glove.6B.100d.txt'
embed_dim = 100 #word vector dim
embedding_matrix = get_embed_mat(EMBEDDING_FILE,max_features,embed_dim)
print(embedding_matrix.shape)

Found 400000 word vectors.
(13732, 100)


In [48]:
model5 = Sequential()
model5.add(Embedding(max_features, embed_dim, input_length=X_train.shape[1],weights=[embedding_matrix],trainable=True))
model5.add(SpatialDropout1D(0.25))
model5.add(Bidirectional(GRU(128,return_sequences=True)))
model5.add(Bidirectional(GRU(64,return_sequences=False)))
model5.add(Dropout(0.5))
model5.add(Dense(num_classes, activation='softmax'))
model5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model5.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 48, 100)           1373200   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 48, 100)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 48, 256)           175872    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128)               123264    
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 645       
Total params: 1,672,981
Trainable params: 1,672,981
Non-trainable params: 0
_________________________________________________________________


In [ ]:
%%time
history5=model5.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=4, batch_size=batch_size, verbose=1)

Train on 124848 samples, validate on 31212 samples
Epoch 1/4
124848/124848 [==============================] - 748s 6ms/step - loss: 0.9932 - acc: 0.5909 - val_loss: 0.8449 - val_acc: 0.6474
Epoch 2/4
124848/124848 [==============================] - 722s 6ms/step - loss: 0.8460 - acc: 0.6487 - val_loss: 0.7959 - val_acc: 0.6731
Epoch 3/4
 29184/124848 [======>.......................] - ETA: 8:31 - loss: 0.7938 - acc: 0.6697

In [ ]:
y_pred5=model5.predict_classes(X_test, verbose=1)